# Project C: Store Sales - Time Series Forecasting

<p>Project members: Juhan Pauklin, Joonas Tiitson, Kristjand Radsin</p>

In [1]:
#Importing modules
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
oil = pd.read_csv("oil.csv")
stores = pd.read_csv("stores.csv")
transactions = pd.read_csv("transactions.csv")
holidays_events = pd.read_csv("holidays_events.csv")

In [3]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [4]:
oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [5]:

# Forward fill missing values
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='ffill')

# Backward fill missing values for the first pass
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='bfill')

# Backward fill again for the last pass, in case the first value is missing
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='bfill')

# If somehow there's still missing values, fill them with the mean
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(value=oil['dcoilwtico'].mean())

In [6]:
oil.describe()

,dcoilwtico
count,1218.000000
mean,67.692159
std,25.629744
min,26.190000
25%,46.422500
50%,53.200000
75%,95.685000
max,110.620000


In [7]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [8]:
merged_train = pd.merge(train, oil, how='left', on='date')
merged_train['dcoilwtico'].fillna(value=oil['dcoilwtico'].mean(), inplace=True)
merged_train

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,93.14
1,1,2013-01-01,1,BABY CARE,0.000,0,93.14
2,2,2013-01-01,1,BEAUTY,0.000,0,93.14
3,3,2013-01-01,1,BEVERAGES,0.000,0,93.14
4,4,2013-01-01,1,BOOKS,0.000,0,93.14
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57


In [9]:
merged_train[merged_train.isna().any(axis=1)]

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico


In [10]:
merged_train = pd.merge(merged_train, stores, how='left', on='store_nbr')
merged_train

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,93.14,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.000,0,93.14,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.000,0,93.14,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.000,0,93.14,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.000,0,93.14,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,47.57,Quito,Pichincha,B,6
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57,Quito,Pichincha,B,6
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57,Quito,Pichincha,B,6
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57,Quito,Pichincha,B,6


In [11]:
merged_train[merged_train.isna().any(axis=1)]

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,type,cluster


In [12]:
merged_train.value_counts('city')

city
Quito            1000296
Guayaquil         444576
Santo Domingo     166716
Cuenca            166716
Manta             111144
Machala           111144
Latacunga         111144
Ambato            111144
Guaranda           55572
Ibarra             55572
Esmeraldas         55572
Babahoyo           55572
Loja               55572
El Carmen          55572
Daule              55572
Playas             55572
Puyo               55572
Quevedo            55572
Cayambe            55572
Riobamba           55572
Salinas            55572
Libertad           55572
dtype: int64

In [13]:
holidays_events['type'].value_counts()

Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: type, dtype: int64

In [14]:
merged_train['holiday'] = False

In [15]:
for ind in holidays_events.index:
    holiday_local = holidays_events['locale_name'][ind]
    row = merged_train[(merged_train['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
    row_i = row.index
    merged_train.loc[row_i, 'holiday'] = True


In [16]:
merged_train.value_counts('holiday')

holiday
False    2987985
True       12903
dtype: int64

In [17]:
merged_train.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,type,cluster,holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,Quito,Pichincha,D,13,False
1,1,2013-01-01,1,BABY CARE,0.0,0,93.14,Quito,Pichincha,D,13,False
2,2,2013-01-01,1,BEAUTY,0.0,0,93.14,Quito,Pichincha,D,13,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,93.14,Quito,Pichincha,D,13,False
4,4,2013-01-01,1,BOOKS,0.0,0,93.14,Quito,Pichincha,D,13,False


In [21]:
merged_train.to_csv("merged_data.csv", index=False) #Writing the merged data into a new file for convenience

In [22]:
merged_train_2 = pd.read_csv("merged_data.csv")

In [23]:
merged_train_2.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,type,cluster,holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,Quito,Pichincha,D,13,False
1,1,2013-01-01,1,BABY CARE,0.0,0,93.14,Quito,Pichincha,D,13,False
2,2,2013-01-01,1,BEAUTY,0.0,0,93.14,Quito,Pichincha,D,13,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,93.14,Quito,Pichincha,D,13,False
4,4,2013-01-01,1,BOOKS,0.0,0,93.14,Quito,Pichincha,D,13,False
